In [ ]:
from websocket import create_connection
import simplejson as json
#from secret import Secret
import pprint
import csv
import pandas as pd
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from keras.layers import LSTM, Dense
import numpy as np

uri = 'wss://stream.data.alpaca.markets/v1beta1/crypto?exchanges=CBSE'
ws = create_connection(uri)

auth_message = {"action":"auth","key": <PLACE KEY HERE>, "secret": <PLACE SECRET KEY HERE>}
ws.send(json.dumps(auth_message))

subscription = {"action":"subscribe","trades":["BTCUSD"]} #Choose what to trade here BTCUSD
ws.send(json.dumps(subscription))

def difference(dataset, interval=1):  
    
    global diff
    
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        value = value[0]
        diff.append(value)
    
    return diff

# frame a sequence as a supervised learning problem
def timeseries_to_supervised(data, lag=1):
	df = DataFrame(data)
	columns = [df.shift(i) for i in range(1, lag+1)]
	columns.append(df)
	df = concat(columns, axis=1)
	df.fillna(0, inplace=True)
	return df

def difference(dataset, interval=1):  
    
    global diff
    
    diff = list()
    for i in range(interval, len(dataset)):
        value = dataset[i] - dataset[i - interval]
        #value = value[0]
        diff.append(value)
    
    return diff

def scale(train, test):
	# fit scaler
	scaler = MinMaxScaler(feature_range=(-1, 1))
	scaler = scaler.fit(train)
	# transform train
	train = train.reshape(train.shape[0], train.shape[1])
	train_scaled = scaler.transform(train)
	# transform test
	test = test.reshape(test.shape[0], test.shape[1])
	test_scaled = scaler.transform(test)
	return scaler, train_scaled, test_scaled

# make a one-step forecast
def forecast_lstm(model, batch_size, X):
	X = X.reshape(1, 1, len(X))
	yhat = model.predict(X, batch_size=batch_size)
	return yhat[0,0]

# inverse scaling for a forecasted value
def invert_scale(scaler, X, value):
	new_row = [x for x in X] + [value]
	array = np.array(new_row)#numpy
	array = array.reshape(1, len(array))
	inverted = scaler.inverse_transform(array)
	return inverted[0, -1]

# invert differenced value
def inverse_difference(history, yhat, interval=1):
	return yhat + history[-interval]

# fit an LSTM network to training data
def fit_lstm(train, batch_size, nb_epoch, neurons):
	X, y = train[:, 0:-1], train[:, -1]
	X = X.reshape(X.shape[0], 1, X.shape[1])
	model = Sequential()
	model.add(LSTM(neurons, batch_input_shape=(batch_size, X.shape[1], X.shape[2]), stateful=True))
	model.add(Dense(1))
	model.compile(loss='mean_squared_error', optimizer='adam')
	
	model.fit(X, y, epochs=1, batch_size=batch_size, verbose=0, shuffle=False)
	model.reset_states()
	return model

while True:
    data = json.loads(ws.recv())
    #pprint.pprint(data[0])
    data_dict = data[0]
    #pprint.pprint(data_dict)
    
    data_df = pd.DataFrame([data_dict])
    
    df_read = pd.read_csv ('Output.csv')
    
    df_read_new = df_read.append(data_df , ignore_index=True)
    
    df_read_new = df_read_new[df_read_new['S'].str.startswith('BTCUSD', na=False)]
    
    df_read_new = df_read_new[["T","S","x","p","s","t","i","tks"]]
    
    #print(df_read_new)
    
    df_read_new.to_csv("Output.csv", index = False)
    
    #print(data_df)
    
    df_read_new = df_read_new["p"]

    raw_values = df_read_new.values

    diff_values = difference(raw_values)

    # transform data to be supervised learning
    supervised = timeseries_to_supervised(diff_values, 1)
    supervised_values = supervised.values

    # split data into train and test-sets
    train, test = supervised_values[0:-12], supervised_values[-12:]

    # transform the scale of the data
    scaler, train_scaled, test_scaled = scale(train, test)

    #train_scaled = tf.expand_dims(train_scaled,1)    
    lstm_model = fit_lstm(train_scaled, 1, 3000, 4)

    # forecast the entire training dataset to build up state for forecasting
    train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)

    # make one-step forecast        
    X, y = test_scaled[0, 0:-1], test_scaled[0, -1]
    yhat = forecast_lstm(lstm_model, 1, X)

    yhat = invert_scale(scaler, X, yhat)

    yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-0)
    
    print("Last trade:", raw_values[-1])
    
    prediction_list = []

    print("Initial prediction:",yhat)
    
    prediction_list.append(yhat)
    
    #### FORECAST FORWARD ####
    
    forecast_length = 5
    
    for i in range(forecast_length):
        
        raw_values = np.append(raw_values, yhat)
        
        diff_values = difference(raw_values)

        # transform data to be supervised learning
        supervised = timeseries_to_supervised(diff_values, 1)
        supervised_values = supervised.values

        # split data into train and test-sets
        train, test = supervised_values[0:-12], supervised_values[-12:]

        # transform the scale of the data
        scaler, train_scaled, test_scaled = scale(train, test)

        #train_scaled = tf.expand_dims(train_scaled,1)    
        lstm_model = fit_lstm(train_scaled, 1, 3000, 4)

        # forecast the entire training dataset to build up state for forecasting
        train_reshaped = train_scaled[:, 0].reshape(len(train_scaled), 1, 1)

        # make one-step forecast        
        X, y = test_scaled[0, 0:-1], test_scaled[0, -1]
        yhat = forecast_lstm(lstm_model, 1, X)

        yhat = invert_scale(scaler, X, yhat)

        yhat = inverse_difference(raw_values, yhat, len(test_scaled)+1-0)

        print("For step", i , "prediction:",yhat)
        
        prediction_list.append(yhat)
        
        if i == forecast_length - 1:
            
            print("All prediction:", prediction_list)
            
            print("Initial prediction:",yhat)
    
    